In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')

# Set display options
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

print("Libraries loaded successfully!")

## 1. Dataset Loading

### 1.1 CICIDS2017 Dataset
Download from: https://www.unb.ca/cic/datasets/ids-2017.html

In [ ]:
# Define data paths
DATA_DIR = Path('../data/datasets')
CICIDS_DIR = DATA_DIR / 'CICIDS2017'
UNSW_DIR = DATA_DIR / 'UNSW-NB15'

# List available files
print("Looking for datasets...")
print(f"\nCICIDS2017 directory: {CICIDS_DIR}")
print(f"UNSW-NB15 directory: {UNSW_DIR}")

# Check if directories exist
if CICIDS_DIR.exists():
    print(f"\nCICIDS2017 files: {list(CICIDS_DIR.glob('*.csv'))}")
else:
    print("\n⚠️ CICIDS2017 directory not found. Please download the dataset.")

if UNSW_DIR.exists():
    print(f"\nUNSW-NB15 files: {list(UNSW_DIR.glob('*.csv'))}")
else:
    print("\n⚠️ UNSW-NB15 directory not found. Please download the dataset.")

In [ ]:
def load_cicids2017(sample_size=None):
    """
    Load CICIDS2017 dataset.
    
    Args:
        sample_size: Number of rows to sample (None for all)
    Returns:
        DataFrame with combined data
    """
    csv_files = list(CICIDS_DIR.glob('*.csv'))
    
    if not csv_files:
        print("No CSV files found. Creating sample data for demonstration...")
        return create_sample_data()
    
    dfs = []
    for file in csv_files:
        print(f"Loading: {file.name}")
        df = pd.read_csv(file, low_memory=False)
        dfs.append(df)
    
    combined = pd.concat(dfs, ignore_index=True)
    
    if sample_size:
        combined = combined.sample(n=min(sample_size, len(combined)), random_state=42)
    
    return combined


def create_sample_data(n_samples=10000):
    """
    Create sample network traffic data for demonstration.
    """
    np.random.seed(42)
    
    # Attack types
    labels = ['BENIGN'] * int(n_samples * 0.7) + \
             ['DoS Hulk'] * int(n_samples * 0.1) + \
             ['PortScan'] * int(n_samples * 0.08) + \
             ['DDoS'] * int(n_samples * 0.05) + \
             ['Bot'] * int(n_samples * 0.03) + \
             ['FTP-Patator'] * int(n_samples * 0.02) + \
             ['SSH-Patator'] * int(n_samples * 0.02)
    
    n = len(labels)
    
    data = {
        'Flow Duration': np.random.exponential(1000000, n),
        'Total Fwd Packets': np.random.poisson(10, n),
        'Total Backward Packets': np.random.poisson(8, n),
        'Total Length of Fwd Packets': np.random.exponential(500, n),
        'Total Length of Bwd Packets': np.random.exponential(400, n),
        'Flow Bytes/s': np.random.exponential(10000, n),
        'Flow Packets/s': np.random.exponential(100, n),
        'Flow IAT Mean': np.random.exponential(10000, n),
        'Flow IAT Std': np.random.exponential(5000, n),
        'Fwd PSH Flags': np.random.binomial(5, 0.3, n),
        'Bwd PSH Flags': np.random.binomial(3, 0.2, n),
        'Fwd Header Length': np.random.randint(20, 100, n),
        'Bwd Header Length': np.random.randint(20, 80, n),
        'Packet Length Mean': np.random.exponential(100, n),
        'Packet Length Std': np.random.exponential(50, n),
        'Packet Length Variance': np.random.exponential(1000, n),
        'SYN Flag Count': np.random.binomial(3, 0.4, n),
        'ACK Flag Count': np.random.binomial(10, 0.6, n),
        'FIN Flag Count': np.random.binomial(2, 0.3, n),
        'RST Flag Count': np.random.binomial(1, 0.1, n),
        'Destination Port': np.random.choice([80, 443, 22, 21, 8080, 3389], n),
        'Protocol': np.random.choice([6, 17, 1], n, p=[0.7, 0.25, 0.05]),
        'Label': labels
    }
    
    df = pd.DataFrame(data)
    np.random.shuffle(df.values)
    
    print(f"Created sample data with {len(df)} records")
    return df


# Load data
df = load_cicids2017(sample_size=50000)
print(f"\nDataset shape: {df.shape}")

## 2. Data Overview

In [ ]:
# Basic info
print("=" * 60)
print("DATASET OVERVIEW")
print("=" * 60)
print(f"\nShape: {df.shape}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1e6:.2f} MB")
print(f"\nColumn types:")
print(df.dtypes.value_counts())

In [ ]:
# Display first rows
df.head(10)

In [ ]:
# Column names
print("Columns:")
for i, col in enumerate(df.columns):
    print(f"{i+1:3d}. {col}")

In [ ]:
# Statistical summary
df.describe()

## 3. Target Variable Analysis

In [ ]:
# Find label column
label_col = None
for col in ['Label', 'label', 'attack_cat', 'Attack']:
    if col in df.columns:
        label_col = col
        break

if label_col:
    print(f"Label column: {label_col}")
    print(f"\nClass distribution:")
    class_dist = df[label_col].value_counts()
    print(class_dist)
    
    print(f"\nClass percentages:")
    print((class_dist / len(df) * 100).round(2))
else:
    print("No label column found!")

In [ ]:
# Visualize class distribution
if label_col:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Bar chart
    class_dist = df[label_col].value_counts()
    colors = plt.cm.Set3(np.linspace(0, 1, len(class_dist)))
    
    axes[0].barh(range(len(class_dist)), class_dist.values, color=colors)
    axes[0].set_yticks(range(len(class_dist)))
    axes[0].set_yticklabels(class_dist.index)
    axes[0].set_xlabel('Count')
    axes[0].set_title('Class Distribution (Count)')
    
    # Pie chart
    axes[1].pie(class_dist.values, labels=class_dist.index, autopct='%1.1f%%',
                colors=colors, startangle=90)
    axes[1].set_title('Class Distribution (%)')
    
    plt.tight_layout()
    plt.savefig('../data/processed/class_distribution.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\n⚠️ Note: Dataset is highly imbalanced - BENIGN class dominates")

## 4. Feature Analysis

In [ ]:
# Identify numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"Numeric features: {len(numeric_cols)}")
print(f"Categorical features: {len(categorical_cols)}")

In [ ]:
# Missing values analysis
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)

missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing %': missing_pct
}).sort_values('Missing Count', ascending=False)

print("Missing Values:")
print(missing_df[missing_df['Missing Count'] > 0])

if missing_df['Missing Count'].sum() == 0:
    print("✅ No missing values found!")

In [ ]:
# Check for infinite values
inf_counts = {}
for col in numeric_cols:
    inf_count = np.isinf(df[col]).sum()
    if inf_count > 0:
        inf_counts[col] = inf_count

if inf_counts:
    print("Columns with infinite values:")
    for col, count in inf_counts.items():
        print(f"  {col}: {count}")
else:
    print("✅ No infinite values found!")

In [ ]:
# Feature distributions
sample_features = ['Flow Duration', 'Total Fwd Packets', 'Flow Bytes/s', 'Packet Length Mean']
sample_features = [f for f in sample_features if f in df.columns][:4]

if sample_features:
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.flatten()
    
    for idx, feature in enumerate(sample_features):
        # Remove infinites and outliers for visualization
        data = df[feature].replace([np.inf, -np.inf], np.nan).dropna()
        q99 = data.quantile(0.99)
        data = data[data <= q99]
        
        axes[idx].hist(data, bins=50, edgecolor='black', alpha=0.7)
        axes[idx].set_xlabel(feature)
        axes[idx].set_ylabel('Frequency')
        axes[idx].set_title(f'Distribution: {feature}')
    
    plt.tight_layout()
    plt.savefig('../data/processed/feature_distributions.png', dpi=150, bbox_inches='tight')
    plt.show()

## 5. Correlation Analysis

In [ ]:
# Select top features for correlation
top_features = numeric_cols[:15] if len(numeric_cols) > 15 else numeric_cols

# Clean data for correlation
corr_df = df[top_features].replace([np.inf, -np.inf], np.nan).dropna()
correlation_matrix = corr_df.corr()

# Plot heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, square=True, linewidths=0.5)
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.savefig('../data/processed/correlation_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Find highly correlated features
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_val = abs(correlation_matrix.iloc[i, j])
        if corr_val > 0.9:
            high_corr_pairs.append({
                'Feature 1': correlation_matrix.columns[i],
                'Feature 2': correlation_matrix.columns[j],
                'Correlation': correlation_matrix.iloc[i, j]
            })

if high_corr_pairs:
    print("Highly correlated feature pairs (|r| > 0.9):")
    high_corr_df = pd.DataFrame(high_corr_pairs).sort_values('Correlation', key=abs, ascending=False)
    print(high_corr_df.to_string(index=False))
else:
    print("No highly correlated pairs found (|r| > 0.9)")

## 6. Attack-Specific Analysis

In [ ]:
if label_col:
    # Compare feature means across attack types
    feature_cols = [c for c in numeric_cols if c != label_col][:10]
    
    attack_stats = df.groupby(label_col)[feature_cols].mean()
    
    print("Mean feature values by attack type:")
    display(attack_stats.round(2))

In [ ]:
if label_col:
    # Box plots for key features by attack type
    key_features = ['Flow Duration', 'Total Fwd Packets', 'Flow Bytes/s']
    key_features = [f for f in key_features if f in df.columns][:3]
    
    if key_features:
        fig, axes = plt.subplots(1, len(key_features), figsize=(15, 5))
        if len(key_features) == 1:
            axes = [axes]
        
        for idx, feature in enumerate(key_features):
            # Clean and cap outliers
            plot_df = df[[label_col, feature]].copy()
            plot_df = plot_df.replace([np.inf, -np.inf], np.nan).dropna()
            q95 = plot_df[feature].quantile(0.95)
            plot_df = plot_df[plot_df[feature] <= q95]
            
            plot_df.boxplot(column=feature, by=label_col, ax=axes[idx])
            axes[idx].set_title(f'{feature} by Attack Type')
            axes[idx].set_xlabel('Attack Type')
            plt.sca(axes[idx])
            plt.xticks(rotation=45, ha='right')
        
        plt.suptitle('')
        plt.tight_layout()
        plt.savefig('../data/processed/features_by_attack.png', dpi=150, bbox_inches='tight')
        plt.show()

## 7. Port Analysis

In [ ]:
# Analyze destination ports
port_col = None
for col in ['Destination Port', 'dst_port', 'Dst Port']:
    if col in df.columns:
        port_col = col
        break

if port_col:
    # Top ports
    top_ports = df[port_col].value_counts().head(20)
    
    plt.figure(figsize=(12, 5))
    top_ports.plot(kind='bar', color='steelblue', edgecolor='black')
    plt.xlabel('Port')
    plt.ylabel('Count')
    plt.title('Top 20 Destination Ports')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('../data/processed/top_ports.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # Port distribution by attack type
    if label_col:
        print("\nPort distribution by attack type:")
        port_attack = df.groupby([label_col, port_col]).size().unstack(fill_value=0)
        print(port_attack.iloc[:, :10])

## 8. Protocol Analysis

In [ ]:
# Analyze protocols
proto_col = None
for col in ['Protocol', 'protocol', 'Proto']:
    if col in df.columns:
        proto_col = col
        break

if proto_col:
    # Map protocol numbers
    proto_map = {6: 'TCP', 17: 'UDP', 1: 'ICMP'}
    df['Protocol_Name'] = df[proto_col].map(proto_map).fillna('Other')
    
    proto_dist = df['Protocol_Name'].value_counts()
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Bar chart
    proto_dist.plot(kind='bar', ax=axes[0], color=['#2ecc71', '#3498db', '#e74c3c', '#95a5a6'])
    axes[0].set_xlabel('Protocol')
    axes[0].set_ylabel('Count')
    axes[0].set_title('Protocol Distribution')
    axes[0].tick_params(axis='x', rotation=0)
    
    # Protocol by attack
    if label_col:
        proto_attack = df.groupby([label_col, 'Protocol_Name']).size().unstack(fill_value=0)
        proto_attack.plot(kind='bar', stacked=True, ax=axes[1])
        axes[1].set_xlabel('Attack Type')
        axes[1].set_ylabel('Count')
        axes[1].set_title('Protocol by Attack Type')
        axes[1].tick_params(axis='x', rotation=45)
        axes[1].legend(title='Protocol')
    
    plt.tight_layout()
    plt.savefig('../data/processed/protocol_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()

## 9. Data Quality Summary

In [ ]:
# Summary report
print("=" * 60)
print("DATA QUALITY SUMMARY")
print("=" * 60)

print(f"\n📊 Dataset Size: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"📁 Memory Usage: {df.memory_usage(deep=True).sum() / 1e6:.2f} MB")

print(f"\n✅ Numeric Features: {len(numeric_cols)}")
print(f"✅ Categorical Features: {len(categorical_cols)}")

missing_total = df.isnull().sum().sum()
print(f"\n{'✅' if missing_total == 0 else '⚠️'} Missing Values: {missing_total}")

inf_total = sum(np.isinf(df[col]).sum() for col in numeric_cols if df[col].dtype in ['float64', 'float32'])
print(f"{'✅' if inf_total == 0 else '⚠️'} Infinite Values: {inf_total}")

if label_col:
    n_classes = df[label_col].nunique()
    print(f"\n🏷️ Target Classes: {n_classes}")
    
    # Class imbalance ratio
    class_dist = df[label_col].value_counts()
    imbalance_ratio = class_dist.max() / class_dist.min()
    print(f"⚖️ Class Imbalance Ratio: {imbalance_ratio:.2f}x")

print("\n" + "=" * 60)
print("RECOMMENDATIONS")
print("=" * 60)
print("\n1. Handle class imbalance (SMOTE, class weights, undersampling)")
print("2. Remove highly correlated features (>0.95)")
print("3. Scale features before ML training")
print("4. Consider log transformation for skewed features")
print("5. Replace infinite values with NaN and impute")

In [ ]:
# Save exploration results
exploration_summary = {
    'n_samples': len(df),
    'n_features': len(df.columns),
    'numeric_features': len(numeric_cols),
    'categorical_features': len(categorical_cols),
    'missing_values': int(missing_total),
    'infinite_values': int(inf_total),
    'n_classes': int(df[label_col].nunique()) if label_col else 0,
    'class_distribution': df[label_col].value_counts().to_dict() if label_col else {}
}

import json
with open('../data/processed/exploration_summary.json', 'w') as f:
    json.dump(exploration_summary, f, indent=2)

print("✅ Exploration summary saved to data/processed/exploration_summary.json")

## 10. Next Steps

Proceed to the next notebooks:
1. **02_feature_engineering.ipynb** - Feature selection and transformation
2. **03_model_training.ipynb** - Train ML models
3. **04_model_evaluation.ipynb** - Evaluate model performance
4. **05_explainability.ipynb** - SHAP analysis